# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [3]:
import os
api_version = "2023-05-15"
deployment = "gpt-35-turbo"
model = "gpt-35-turbo"

# 更换为自己的 Serp API KEY
os.environ["SERPAPI_API_KEY"] = "4ca33e22bb964e08c559f4ea3b69324626ad68a574381de788894deaab9c0fd1"

In [40]:
from langchain_openai import AzureChatOpenAI, AzureOpenAI
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = AzureOpenAI(
    azure_deployment=deployment,
    api_version=api_version
)

In [41]:
from langchain_community.utilities.google_serper import GoogleSerperAPIWrapper

# 实例化查询工具
search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [42]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [43]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.invoke(
    "成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'code': 'OperationNotSupported', 'message': 'The completion operation does not work with the specified model, gpt-35-turbo. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.'}}

In [38]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.invoke(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output: Yes.

In [7]:
# Reason-only 答案（错误）
print(llm("成都举办的大运会是第几届大运会？"))



成都举办的是第十一届大运会。


In [8]:
# Reason-only 答案（错误）
print(llm("2023年大运会举办地在哪里？"))



2023年大运会将在日本东京举办。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [9]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [10]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [11]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.run(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: What is the English translation of "2023年大运会"?
Intermediate answer: No good search result found
The phrase "2023年大运会" translates to "2023 Universiade" in English. The Universiade is an international multi-sport event, organized for university athletes by the International University Sports Federation (FISU).

Follow up: Where is the 2023 Universiade being held?
Intermediate answer: Lake Placid 2023 FISU World University Games Winter, 12-22 January. Welcome back to legendary Lake Placid, where many Universiade and Olympic Legends were born.
So the final answer is: Lake Placid, New York, United States

> Finished chain.


'Lake Placid, New York, United States'